# This file is for inference
#### postprocessing + ensemble 3 models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g5wPfsX32OLZR5G8iqg_C54RnxIiAtp6_Uz8gTajht_PDY78QHnKAo
Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
from torch.utils.data import DataLoader
import torch
from torch.utils.data import Dataset
import torch.nn as nn

In [ ]:
class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

In [ ]:
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

### Model Structures

In [ ]:
########### model1 ###############
class BasicBlock01(nn.Module):
    def __init__(self, input_dim, output_dim, p=0.5):
        super(BasicBlock01, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.LeakyReLU(),
            nn.Dropout(p),
        )

    def forward(self, x):
        x = self.block(x)
        return x

class Classifier01(nn.Module):
    def __init__(self, input_dim=429, output_dim=39, hidden_layers=5, hidden_dim=2048):
        super(Classifier01, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock01(input_dim, hidden_dim),
            *[BasicBlock01(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
########### model2 ###############
class BasicBlock02(nn.Module):
    def __init__(self, input_dim, output_dim, p=0.5):
        super(BasicBlock02, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.Dropout(p),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x

class Classifier02(nn.Module):
    def __init__(self, input_dim=429, output_dim=39, hidden_layers=5, hidden_dim=2048):
        super(Classifier02, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock02(input_dim, hidden_dim),
            *[BasicBlock02(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
########### model3 ###############
class BasicBlock03(nn.Module):
    def __init__(self, input_dim, output_dim, p=0.5):
        super(BasicBlock03, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.ReLU(),
            nn.Dropout(p),
        )

    def forward(self, x):
        x = self.block(x)
        return x

class Classifier03(nn.Module):
    def __init__(self, input_dim=429, output_dim=39, hidden_layers=6, hidden_dim=2048):
        super(Classifier03, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock03(input_dim, hidden_dim),
            *[BasicBlock03(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

### Testing

In [ ]:
def inference(model, device):
    predict = []
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs = data
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

            for y in test_pred.cpu().numpy():
                predict.append(y)
    return predict

In [ ]:
def postprocess(predict):
    cur_phone = predict[0]
    count = 1

    for i in range(1,len(predict)):
        if predict[i] == cur_phone:
            count += 1
        else:
            if count>3:
                count = 1
            else:
                if predict[i]==predict[i-count-1]:
                    predict[i-count:i] = predict[i]
                    tmp = 0
                    k = i-count-2
                    while predict[k] == predict[i]:
                        tmp += 1
                        k -= 1
                    count += tmp
                else:
                    count = 1
            cur_phone = predict[i]
    return predict

In [ ]:
def ensemble(preds):
    print('ensemble...')
    new_val = []
    for i in range(test.shape[0]):
        x = preds[:,i]
        m = np.bincount(x).argmax()
        new_val.append(m)
    
    return new_val

In [ ]:
def main():
    device = get_device()
    #model1
    print('Loading model 1...')
    net01 = Classifier01().to(device)
    net01.load_state_dict(torch.load('/content/drive/MyDrive/checkpoints/model_01.ckpt'))#YOUR MODEL1 PATH HERE
    #model2
    print('Loading model 2...')
    net02 = Classifier02().to(device)
    net02.load_state_dict(torch.load('/content/drive/MyDrive/checkpoints/model_02.ckpt'))#YOUR MODEL2 PATH HERE
    #model3
    print('Loading model 3...')
    net03 = Classifier03().to(device)
    net03.load_state_dict(torch.load('/content/drive/MyDrive/checkpoints/model_03.ckpt'))#YOUR MODEL3 PATH HERE
    
    print('predicting using model 1...')
    f1 = inference(net01, device)
    f1 = postprocess(np.array(f1))
    print('predicting using model 2...')
    f2 = inference(net02, device)
    f2 = postprocess(np.array(f2))
    print('predicting using model 3...')
    f3 = inference(net03, device)
    f3 = postprocess(np.array(f3))

    new_val = ensemble(np.array([f1,f2,f3]))

    print('writing prediction to output.csv.')
    with open("output.csv", 'w') as f:
        f.write('Id,Class\n')
        for i, y in  enumerate(new_val):
            f.write('{},{}\n'.format(i, y))

In [ ]:
BATCH_SIZE = 256
data_root='/content/drive/MyDrive/timit_11/'
test = np.load(data_root + 'test_11.npy')
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
main()

Loading model 1...
Loading model 2...
Loading model 3...
predicting using model 1...
predicting using model 2...
predicting using model 3...
ensemble...
writing prediction to output.csv.
